# Прогноз невыполнения кредитных обязательств

## 1. Подключение библиотек и скриптов

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

In [2]:
RANDOM_STATE = 64
DATA_PATH = Path('./data')

## 2. Загрузка данных

**Описание датасета**

* Home Ownership - домовладение
* Annual Income - годовой доход
* Years in current job - количество лет на текущем месте работы
* Tax Liens - налоговые обременения
* Number of Open Accounts - количество открытых счетов
* Years of Credit History - количество лет кредитной истории
* Maximum Open Credit - наибольший открытый кредит
* Number of Credit Problems - количество проблем с кредитом
* Months since last delinquent - количество месяцев с последней просрочки платежа
* Bankruptcies - банкротства
* Purpose - цель кредита
* Term - срок кредита
* Current Loan Amount - текущая сумма кредита
* Current Credit Balance - текущий кредитный баланс
* Monthly Debt - ежемесячный долг
* Credit Default - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [3]:
data = pd.read_csv(DATA_PATH / 'train.csv')
data.sample(5)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
2027,Home Mortgage,NaN,6 years,0.0,14.0,17.4,305536.0,1.0,NaN,1.0,debt consolidation,Short Term,246048.0,153311.0,20562.0,NaN,0
5764,Home Mortgage,1313242.0,< 1 year,0.0,25.0,15.8,750112.0,0.0,13.0,0.0,debt consolidation,Short Term,254694.0,277913.0,16306.0,7250.0,1
3231,Rent,582369.0,3 years,0.0,8.0,19.6,226798.0,0.0,35.0,0.0,debt consolidation,Long Term,232342.0,166098.0,9910.0,688.0,0
7216,Rent,616569.0,< 1 year,0.0,6.0,25.9,231748.0,0.0,33.0,0.0,debt consolidation,Short Term,238128.0,153102.0,12177.0,728.0,0
7444,Home Mortgage,1515592.0,10+ years,0.0,21.0,29.8,500544.0,1.0,NaN,1.0,debt consolidation,Short Term,370722.0,332861.0,23492.0,686.0,0


## 3. EDA и очистка данных

### 3.1. Анализ целевой переменной

### 3.2. Анализ признаков

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Home Ownership                7500 non-null   object 
 1   Annual Income                 5943 non-null   float64
 2   Years in current job          7129 non-null   object 
 3   Tax Liens                     7500 non-null   float64
 4   Number of Open Accounts       7500 non-null   float64
 5   Years of Credit History       7500 non-null   float64
 6   Maximum Open Credit           7500 non-null   float64
 7   Number of Credit Problems     7500 non-null   float64
 8   Months since last delinquent  3419 non-null   float64
 9   Bankruptcies                  7486 non-null   float64
 10  Purpose                       7500 non-null   object 
 11  Term                          7500 non-null   object 
 12  Current Loan Amount           7500 non-null   float64
 13  Cur

В датасете есть пропуски по столбцам:
* Annual Income
* Years in current job
* Months since last delinquent
* Bankruptcies
* Credit Score

In [5]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Annual Income,5943.0,1.366392e+06,8.453392e+05,164597.0,844341.0,1168386.0,1640137.00,1.014934e+07
Tax Liens,7500.0,3.013333e-02,2.716035e-01,0.0,0.0,0.0,0.00,7.000000e+00
Number of Open Accounts,7500.0,1.113093e+01,4.908924e+00,2.0,8.0,10.0,14.00,4.300000e+01
Years of Credit History,7500.0,1.831747e+01,7.041946e+00,4.0,13.5,17.0,21.80,5.770000e+01
Maximum Open Credit,7500.0,9.451537e+05,1.602622e+07,0.0,279229.5,478159.0,793501.50,1.304726e+09
Number of Credit Problems,7500.0,1.700000e-01,4.985978e-01,0.0,0.0,0.0,0.00,7.000000e+00
Months since last delinquent,3419.0,3.469260e+01,2.168881e+01,0.0,16.0,32.0,50.00,1.180000e+02
Bankruptcies,7486.0,1.171520e-01,3.471917e-01,0.0,0.0,0.0,0.00,4.000000e+00
Current Loan Amount,7500.0,1.187318e+07,3.192612e+07,11242.0,180169.0,309573.0,519882.00,1.000000e+08
Current Credit Balance,7500.0,2.898332e+05,3.178714e+05,0.0,114256.5,209323.0,360406.25,6.506797e+06


### 3.3. Анализ зависимости таргета от фичей

## 4: Очистка данных и Feature engineering

## 5. Разбиение на train / test

## 6. Построение модели

## 7. Оценка финальной модели